In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('missing').getOrCreate()

In [3]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/ContainsNull.csv,ContainsNull.csv,61
dbfs:/FileStore/tables/Spark_Essentials-5d27c.dbc,Spark_Essentials-5d27c.dbc,1414841
dbfs:/FileStore/tables/appl_stock.csv,appl_stock.csv,143130
dbfs:/FileStore/tables/cogsley_clients.csv,cogsley_clients.csv,384219
dbfs:/FileStore/tables/cogsley_sales.csv,cogsley_sales.csv,2176442
dbfs:/FileStore/tables/people.json,people.json,73
dbfs:/FileStore/tables/sales_info.csv,sales_info.csv,196
dbfs:/FileStore/tables/state_info.csv,state_info.csv,2778


In [4]:
path = 'FileStore/tables/ContainsNull.csv'

In [5]:
df = spark.read.csv(path,inferSchema=True,header = True)

In [6]:
df.show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [7]:
df.na.drop().show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [8]:
df.na.drop(thresh=2).show() # drops anyrow with more then 2 missing values

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [9]:
df.na.drop(how='any').show() # drops in any row has missing values

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [10]:
df.na.drop(subset = ['Sales']).show() # only drops missing value from sales column

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [11]:
df.na.fill('fill').show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| fill| null|
emp3| fill|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [12]:
df.na.fill(0).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| 0.0|
emp2| null| 0.0|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [13]:
df.na.fill('missing',subset =['Name']).show()

+----+-------+-----+
 Id| Name|Sales|
+----+-------+-----+
emp1| John| null|
emp2|missing| null|
emp3|missing|345.0|
emp4| Cindy|456.0|
+----+-------+-----+

In [14]:
# filling missing data with mean value
from pyspark.sql.functions import mean

In [15]:
mean_sales = df.select(mean(df['Sales'])).collect()

In [16]:
mean_sales

Out[ 27 ]: [Row(avg(Sales)=400.5)]

In [17]:
mean_sales[0]

Out[ 28 ]: Row(avg(Sales)=400.5)

In [18]:
mean_sales[0][0]

Out[ 29 ]: 400.5

In [19]:
mean_sales=mean_sales[0][0]

In [20]:
df.na.fill(mean_sales,subset=['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+